Notebook for the GoogleDecimeterChallenge https://www.kaggle.com/competitions/smartphone-decimeter-2023


In [3]:
#reading data
import numpy as np 
import pandas as pd
from tqdm import tqdm, trange

INPUT_PATH = 'sdc2023/'

df_sample_trail_gnss = pd.read_csv(INPUT_PATH + "train/2020-06-25-00-34-us-ca-mtv-sb-101/pixel4/device_gnss.csv")
df_sample_trail_gt = pd.read_csv(INPUT_PATH + "train/2020-06-25-00-34-us-ca-mtv-sb-101/pixel4/ground_truth.csv")
sample_submission = pd.read_csv(INPUT_PATH + "sample_submission.csv")